In [81]:
import os

In [82]:
# os.chdir("../")

In [83]:
%pwd

'd:\\chicken_fecal_clasification'

In [84]:
import tensorflow as tf


In [85]:
model = tf.keras.models.load_model(r'artifacts\training\model.h5')

In [86]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data:Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [87]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [88]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=r"artifacts\training\model.h5",
            training_data=r"artifacts\data_ingestion\Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [89]:
from urllib.parse import urlparse

In [90]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    

In [91]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-05-29 02:36:58,860] {d:\chicken_fecal_clasification\src\cnnClassifier\utils\common.py:24} INFO - yaml file: config\config.yaml read successfully
[2024-05-29 02:36:58,864] {d:\chicken_fecal_clasification\src\cnnClassifier\utils\common.py:24} INFO - yaml file: params.yaml read successfully
[2024-05-29 02:36:58,866] {d:\chicken_fecal_clasification\src\cnnClassifier\utils\common.py:42} INFO - Created directory: artifacts
Found 116 images belonging to 2 classes.
8/8 [==============================] - 4s 266ms/step - loss: 0.9535 - accuracy: 0.8879
[2024-05-29 02:37:03,662] {d:\chicken_fecal_clasification\src\cnnClassifier\utils\common.py:55} INFO - Data saved to json file: scores.json
